<a href="https://colab.research.google.com/github/JeongHanJun/Colab/blob/master/Keras_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
'''
    Input -> 784 (28x28) dimentional vectors
    
    Dense -> 64 units / relu activation
    Dense -> 64 units / relu activation
    Dense -> 10 units / softmax activation

    Output -> logits of a probability distribution over 10 classes
'''
inputs = keras.Input(shape = (784,))

img_inputs = keras.Input(shape = (32, 32, 3))
inputs.shape

In [ ]:
inputs.dtype

In [ ]:
dense = layers.Dense(64, activation = 'relu')
x = dense(inputs)

In [ ]:
x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(10)(x)

In [ ]:
models = keras.Model(inputs = inputs, outputs = outputs, name = "mnist_data")

In [ ]:
models.summary()

In [ ]:
keras.utils.plot_model(models, 'my_first_model.png')

In [ ]:
keras.utils.plot_model(models, 'my_first_model_with_shape_info.png', show_shapes = True)

In [ ]:
# 트트 테테
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

models.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.RMSprop(),
    metrics = ['accuracy'],
)

history = models.fit(x_train, y_train, batch_size = 64, epochs = 2, validation_split = 0.2)

test_scores = models.evaluate(x_test, y_test, verbose = 2)
print('Test loss = ', test_scores[0])
print('Test accuracy = ', test_scores[1])

In [ ]:
models.save('path_to_my_model')
del models
my_model = keras.models.load_model('path_to_my_model')

In [ ]:
encoder_input = keras.Input(shape = (28, 28, 1), name = 'img')

L1 = layers.Conv2D(16, 3, activation = 'relu')(encoder_input)
L2 = layers.Conv2D(32, 3, activation = 'relu')(L1)
L3 = layers.MaxPooling2D(3)(L2)

L4 = layers.Conv2D(32, 3, activation = 'relu')(L3)
L5 = layers.Conv2D(16, 3, activation = 'relu')(L4)

encoder_output = layers.GlobalMaxPooling2D()(L5)

encoder = keras.Model(encoder_input, encoder_output, name = 'encoder')
encoder.summary()

In [ ]:
L1 = layers.Reshape( (4, 4, 1) )(encoder_output)
L2 = layers.Conv2DTranspose(16, 3, activation = 'relu')(L1)
L3 = layers.Conv2DTranspose(32, 3, activation = 'relu')(L2)

L4 = layers.UpSampling2D(3)(L3)
L5 = layers.Conv2DTranspose(16, 3, activation = 'relu')(L4)

decoder_output = layers.Conv2DTranspose(1, 3, activation = 'relu')(L5)

autoencoder = keras.Model(encoder_input, decoder_output, name = 'autoencoder')
autoencoder.summary()

In [ ]:
encoder_input = keras.Input(shape = (28, 28, 1), name = 'original_img')
x = layers.Conv2D(16, 3, activation="relu")(encoder_input)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

decoder_input = keras.Input(shape=(16,), name="encoded_img")
x = layers.Reshape((4, 4, 1))(decoder_input)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu")(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation="relu")(x)

decoder = keras.Model(decoder_input, decoder_output, name="decoder")
decoder.summary()

autoencoder_input = keras.Input(shape=(28, 28, 1), name="img")
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = keras.Model(autoencoder_input, decoded_img, name="autoencoder")
autoencoder.summary()

In [ ]:
def get_model():
    inputs = keras.Input(shape = (128,))
    outputs = layers.Dense(1)(inputs)
    return keras.Model(inputs, outputs)

model1 = get_model()
model2 = get_model()
model3 = get_model()

inputs = keras.Input(shape = (128, ))
y1 = model1(inputs)
y2 = model1(inputs)
y3 = model1(inputs)
outputs = layers.average([y1, y2, y3])
ensemble_model = keras.Model(inputs = inputs, outputs = outputs)

## Multiple Input / Output Model

* 고객의 티켓을 발급할때, 우선순위를 지정하고, 올바른 부서로 라우팅해주는 시스템의 모델을 만든다고 가정

- 모델에 필요한 입력 3가지
    1. 티켓 제목( text input )
    2. 티켓의 텍스트 본문( text input )
    3. 사용자가 추가한 모든 태그( Category input )

- 모델의 출력 2가지
    1. 0과 1 사이의 우선순위 점수( 스칼라 시그모이드 출력)
    2. 티켓을 처리해야 하는 부서(부서 집합에 대한 softmax 출력)

위에 대한 내용을 Functional API를 통해 모델을 빌드한다면 아래와 같다


In [ ]:
tags = 12
words = 10000
departments = 4

title_input = keras.Input(
    shape = (None, ), name = 'title'
)
body_input = keras.Input(shape = (None, ), name = 'body')
tags_input = keras.Input(
    shape = (tags, ),
    name = 'tag'
)

title_features = layers.Embedding(words, 64)(title_input)
body_features = layers.Embedding(words, 64)(body_input)
title_features = layers.LSTM(128)(title_features)
body_features = layers.LSTM(32)(body_features)

x = layers.concatenate( [title_features, body_features, tags_input])

priority_pred = layers.Dense(1, name = 'priority')(x)
department_pred = layers.Dense(departments, name = 'department')(x)

model = keras.Model(
    inputs = [title_input, body_input, tags_input],
    outputs = [priority_pred, department_pred]
)

In [ ]:
keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes = True)

In [ ]:
model.compile(
    optimizer = keras.optimizers.RMSprop(1e-3),
    loss = [
            keras.losses.BinaryCrossentropy(from_logits = True),
            keras.losses.CategoricalCrossentropy(from_logits = True)
    ],
    loss_weights = [1.0, 0.2]
)

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "priority": keras.losses.BinaryCrossentropy(from_logits=True),
        "department": keras.losses.CategoricalCrossentropy(from_logits=True),
    },
    loss_weights={"priority": 1.0, "department": 0.2},
)

In [ ]:
inputs = keras.Input(shape=(32, 32, 3), name="img")
x = layers.Conv2D(32, 3, activation="relu")(inputs)
x = layers.Conv2D(64, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(64, 3, activation="relu")(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model = keras.Model(inputs, outputs, name="toy_resnet")
model.summary()

In [ ]:
# 위의 모델 도식화
keras.utils.plot_model(model, 'mini_ResNet.png', show_shapes = True)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"],
)
model.fit(x_train[:1000], y_train[:1000], batch_size=64, epochs=1, validation_split=0.2)

## 공유 레이어
- Functional API의 장점중 하나인 공유 레이어를 사용하는 모델
- 동일한 모델에서 여러번 재사용되는 레이어 인스턴스이다.
- 서로 다른 입력간에 정보를 공유할 수 있다.

In [ ]:
shared_embedding = layers.Embedding(1000, 128)

text_input_a = keras.Input(shape=(None,), dtype="int32")
text_input_b = keras.Input(shape=(None,), dtype="int32")

encoded_input_a = shared_embedding(text_input_a)
encoded_input_b = shared_embedding(text_input_b)

## 레이어 그래프에서의 노드 추출 및 재사용
- 계층의 그래프는 정적 데이터 구조이므로 액세스하고 검사할 수 있다.
- 중간 레이어 ( 그래프의 Node )의 활성화에 엑세스 하여 다른 곳에서 재사용할 수 있음을 의미한다.
- 아래는 ImageNet에서 사전 훈련된 가중치가 있는 VGG19모델을 통한 예시이다.

In [ ]:
vgg19 = tf.keras.applications.VGG19()

In [ ]:
features_list = [layer.output for layer in vgg19.layers]

In [ ]:
feat_extraction_model = keras.Model(inputs = vgg19.input, outputs = features_list)

img = np.random.random( (1, 224, 224, 3)).astype('float32')
extracted_features = feat_extraction_model(img)

# tensorflow.keras 안에 포함된 다양한 기본 제공 레이어들
1. Convolution Layer
    - Conv1D, Conv2D, Conv3D, Conv2DTranspose

2. Pooling Layer
    - MaxPooling1D, MaxPooling2D, MaxPooling3D, AveragePooling1D

3. RNN Layer
    - GRU, LSTM, ConvLSTM2D

4. BatchNormalization, Dropout, Embedding, etc

다양한 내용들이 제공되는데, 만약 나에게 필요한 것을 찾지 못한다면?
-> 고유한 레이어를 만들어 API를 확장하고, Layer 클래스를 하위 Layer클래스로 만들고 구현하면 된다.

# tensorflow.keras.layers.Dense 기본 구현

In [ ]:
class CustomDense(layers.Layer):
    def __init__(self, units = 32):
        super(CustomDense, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape = (input_shape[-1], self.units),
            initializer = 'random_normal',
            trainable = True
        )
        self.b = self.add_weight(
            shape = (self.units, ), initializer = 'random_normal', trainable = True
        )
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

inputs = keras.Input( (4,) )
outputs= CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)

In [ ]:
class CustomDense(layers.Layer):
    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    # get_config 메서드
    # 사용자 지정 레이어의 직렬화는 지원, 레이어 인스턴스의 생성자 인수를 반환한다.
    def get_config(self):
        return {"units": self.units}


inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)
config = model.get_config()

new_model = keras.Model.from_config(config, custom_objects={"CustomDense": CustomDense})

In [ ]:
inputs = keras.Input(shape=(32,))
x = layers.Dense(64, activation='relu')(inputs)
outputs = layers.Dense(10)(x)
mlp = keras.Model(inputs, outputs)

In [ ]:
class MLP(keras.Model):

  def __init__(self, **kwargs):
    super(MLP, self).__init__(**kwargs)
    self.dense_1 = layers.Dense(64, activation='relu')
    self.dense_2 = layers.Dense(10)

  def call(self, inputs):
    x = self.dense_1(inputs)
    return self.dense_2(x)

# Instantiate the model.
mlp = MLP()
# Necessary to create the model's state.
# The model doesn't have a state until it's called at least once.
_ = mlp(tf.zeros((1, 32)))

In [ ]:
units = 32
timesteps = 10
input_dim = 5

# Define a Functional model
inputs = keras.Input((None, units))
x = layers.GlobalAveragePooling1D()(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


class CustomRNN(layers.Layer):
    def __init__(self):
        super(CustomRNN, self).__init__()
        self.units = units
        self.projection_1 = layers.Dense(units=units, activation="tanh")
        self.projection_2 = layers.Dense(units=units, activation="tanh")
        # Our previously-defined Functional model
        self.classifier = model

    def call(self, inputs):
        outputs = []
        state = tf.zeros(shape=(inputs.shape[0], self.units))
        for t in range(inputs.shape[1]):
            x = inputs[:, t, :]
            h = self.projection_1(x)
            y = h + self.projection_2(state)
            state = y
            outputs.append(y)
        features = tf.stack(outputs, axis=1)
        print(features.shape)
        return self.classifier(features)


rnn_model = CustomRNN()
_ = rnn_model(tf.zeros((1, timesteps, input_dim)))

In [ ]:
units = 32
timesteps = 10
input_dim = 5
batch_size = 16


class CustomRNN(layers.Layer):
    def __init__(self):
        super(CustomRNN, self).__init__()
        self.units = units
        self.projection_1 = layers.Dense(units=units, activation="tanh")
        self.projection_2 = layers.Dense(units=units, activation="tanh")
        self.classifier = layers.Dense(1)

    def call(self, inputs):
        outputs = []
        state = tf.zeros(shape=(inputs.shape[0], self.units))
        for t in range(inputs.shape[1]):
            x = inputs[:, t, :]
            h = self.projection_1(x)
            y = h + self.projection_2(state)
            state = y
            outputs.append(y)
        features = tf.stack(outputs, axis=1)
        return self.classifier(features)


# Note that you specify a static batch size for the inputs with the `batch_shape`
# arg, because the inner computation of `CustomRNN` requires a static batch size
# (when you create the `state` zeros tensor).
inputs = keras.Input(batch_shape=(batch_size, timesteps, input_dim))
x = layers.Conv1D(32, 3)(inputs)
outputs = CustomRNN()(x)

model = keras.Model(inputs, outputs)

rnn_model = CustomRNN()
_ = rnn_model(tf.zeros((1, 10, 5)))